In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# !pip3 install catboost
from catboost import CatBoostRegressor as cbr


In [2]:
df_train = pd.read_csv('train_int.csv', sep=',', index_col=0) 
df_test = pd.read_csv('test_int.csv', sep=',', index_col=0)

In [3]:
# raw train data
pd.read_csv('train.csv', sep=',').head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [4]:
# raw test data
pd.read_csv('train.csv', sep=',').head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [5]:
df_train.isnull().sum().sum()
df_test.isnull().sum().sum()
df_train_idx = df_train.index
df_test_idx = df_test.index
# df_train['isTrain'] = True
# df_test['isTrain'] = False
traintest = pd.concat([df_train, df_test], axis = 0)

import re
cat_pattern = re.compile("^cat([1-9]|[1-9][0-9]|[1-9][0-9][0-9])$")
cont_pattern = re.compile("^cont([1-9]|[1-9][0-9]|[1-9][0-9][0-9])$")

cat_col = sorted([cat for cat in traintest.columns if 'cat' in cat], key = lambda s: int(s[3:]))
cat_index = [i for i in range(0,len(traintest.columns)-1) if cat_pattern.match(traintest.columns[i])]
cont_col = sorted([cont for cont in traintest.columns if 'cont' in cont], key = lambda s: int(s[4:]))
cont_index = [i for i in range(0,len(traintest.columns)-1) if cont_pattern.match(traintest.columns[i])]
features = cat_col + cont_col


feats_counts = traintest[cat_col].nunique(dropna = False)
# if data is cat 
def cat_to_numbers(Y):
        lenght_Y =len(Y)
        unqiue_Y = np.unique(Y)
        lenght_unqiue_Y = len(unqiue_Y)
        
        cat_to_numbers_Y = []
        for idx in range(len(Y)):
            cat_to_numbers_Y.append(np.where(unqiue_Y == Y[idx])[0][0])
        return cat_to_numbers_Y
#sample
# for col in cat_col:
#     df_train[col] = pd.Series(cat_to_numbers(df_train[col]))
   

In [6]:
df_train.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss,isTrain
0,1,0,1,0,1,0,0,0,0,1,...,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18,True
1,2,0,1,0,0,0,0,0,0,1,...,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60,True
2,5,0,1,0,0,1,0,0,0,1,...,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09,True
3,10,1,1,0,1,0,0,0,0,1,...,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85,True
4,11,0,1,0,1,0,0,0,0,1,...,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85,True


In [7]:

df_test.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,isTrain
0,4,0,1,0,1,0,0,0,0,1,...,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562,False
1,6,0,1,0,0,0,0,0,0,1,...,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045,False
2,9,0,1,0,0,1,0,0,0,1,...,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232,False
3,12,1,1,0,1,0,0,0,0,1,...,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955,False
4,15,0,1,0,1,0,0,0,0,1,...,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823,False


In [8]:
from dask_ml.datasets import make_classification
from dask_ml.wrappers import Incremental
from sklearn.linear_model import SGDClassifier
from dask_ml.linear_model import LogisticRegression
X, y = make_classification(chunks=25)
estimator = SGDClassifier(random_state=10, max_iter=100)
clf = Incremental(estimator)

In [9]:
loss = df_train['loss']
lenght_y =len(df_train['loss'])
y = np.array([int(value // 500) for value in loss])

y

array([ 4,  2,  6, ..., 11,  3,  9])

In [10]:

X = df_train.drop('id', axis = 1)[features]


from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8)


half_data = len(X_train) // 2
from model import status

In [11]:
clf.fit(X_train[:half_data], y_train[:half_data], classes=y)
clf.score(X_validation,y_validation)


0.11929163126593033

In [12]:
clf.fit(X_train[half_data:], y_train[half_data:], classes=y)
clf.score(X_validation,y_validation)

0.1429481733220051

In [ ]:
status(clf,X_train, y_train, X_validation,y_validation,y, 'all data' )

all data -: training accuracy=14.20%, test accuracy=14.29%
